## Imports

In [1]:
import jieba, json, os, re, sys, time
from datetime import datetime
import numpy as np
import tensorboardX
import torch
import torch.nn as nn
import shutil

## Functions

In [2]:
from fields import Field, Parms, Semantic, Vocab, _make_vocab
from utils import *
from functions import *

from nlp_db import nlp_db

from model_class import NLU_Classify

semantic = Semantic()
args = Parms()
vocab = Vocab(semantic)

args.manual_log = './manualog_transfromer1.log'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Dialog Task

### Data 

#### Data Files

In [ ]:
# get train, test file names
rel_path = "../dialog_db/chinese_chatbot_corpus-master/clean_chat_corpus/"
cfiles = pathFiles(rel_path)
[print(file) for file in cfiles]

botTrainFile = cfiles[1]

#### Vocab Preprocess

In [ ]:
args.vocab_path = os.path.abspath(os.path.join(rel_path, 'vocab.txt'))

def make_chatbot_vocab(file, vocab_path, thres = 2):
    word_dict = {}
    with open(file, "r", encoding='utf-8') as f:
        cnt = 0
        for l in f.readlines():
            for token in list(jieba.cut(l.strip().replace('\t',""))):
                if token not in word_dict:
                    word_dict[token] = 0
                else:
                    word_dict[token] += 1

    if not os.path.isfile(vocab_path):
        open(vocab_path,'a').close()

    with open(vocab_path, 'w') as f:
        for k, v in word_dict.items():
            if v > thres:
                print(k, file=f)

In [12]:
os.remove(args.vocab_path)
if not os.path.isfile(args.vocab_path):
    make_chatbot_vocab(botTrainFile, args.vocab_path)

try:
    vocab.load(args.vocab_path)
except:
    print("Vocab not loaded")
vocab.size, vocab.__getitem__('吃'), vocab.__getitem__(
    '<pad>'), vocab.__getitem__('<unk>'), vocab.__getitem__('<sos>')

(32707, 264, 0, 3, 1)

#### Data Process => Model Parms Get

In [27]:
def get_max_sent_len(file):
    with open(file, "r", encoding='utf-8') as f:
        maxlen, sent_count = 0, 0
        for l in f.readlines():
            maxlen = max([maxlen, max([len(sent) for sent in l.split()])])
            sent_count += 1
    
    return maxlen, sent_count

args.max_sent_len, args.sent_count = get_max_sent_len(botTrainFile)

In [ ]:
args.batch_size = 1000
dirrm(args)

### Define the Model

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output


ModuleNotFoundError: No module named 'torch'

In [19]:
transformer_model = nn.Transformer(nhead=8, num_encoder_layers=6)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

In [21]:
out.shape

torch.Size([20, 32, 512])

In [ ]:
!ls -lc /home/ubuntu/Studio/tutorials/tutorials-master/